## Libararies

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
# from imblearn.over_sampling import SMOTE
# from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc

## Load Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wordsforthewise/lending-club")

print("Path to dataset files:", path)


100%|██████████| 1.26G/1.26G [00:11<00:00, 117MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/wordsforthewise/lending-club/versions/3


In [3]:
# Use correct file path and filename
filepath = os.path.join(path, "accepted_2007_to_2018Q4.csv.gz")

df = pd.read_csv(filepath,low_memory=False)
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


## Filter out relevent columns for our research question

In [4]:
relevant_columns = [
    # Loan Characteristics
    "loan_amnt",
    "term",
    "int_rate",
    "grade",
    "purpose",
    # Borrower's Financials & History
    "annual_inc",
    "dti",
    "delinq_2yrs",
    "inq_last_6mths",
    "home_ownership",
    "emp_length",
    "fico_range_low",
    "fico_range_high",
    "issue_d",
    "earliest_cr_line",
    "open_acc",
    "pub_rec",
    "revol_bal",
    "revol_util",
    "total_acc",
    "verification_status",
    "application_type",
    "addr_state",
    # Target Variable
    "loan_status",
]
df = df[relevant_columns]
df.head()

,loan_amnt,term,int_rate,grade,purpose,annual_inc,dti,delinq_2yrs,inq_last_6mths,home_ownership,...,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,verification_status,application_type,addr_state,loan_status
0,3600.0,36 months,13.99,C,debt_consolidation,55000.0,5.91,0.0,1.0,MORTGAGE,...,Aug-2003,7.0,0.0,2765.0,29.7,13.0,Not Verified,Individual,PA,Fully Paid
1,24700.0,36 months,11.99,C,small_business,65000.0,16.06,1.0,4.0,MORTGAGE,...,Dec-1999,22.0,0.0,21470.0,19.2,38.0,Not Verified,Individual,SD,Fully Paid
2,20000.0,60 months,10.78,B,home_improvement,63000.0,10.78,0.0,0.0,MORTGAGE,...,Aug-2000,6.0,0.0,7869.0,56.2,18.0,Not Verified,Joint App,IL,Fully Paid
3,35000.0,60 months,14.85,C,debt_consolidation,110000.0,17.06,0.0,0.0,MORTGAGE,...,Sep-2008,13.0,0.0,7802.0,11.6,17.0,Source Verified,Individual,NJ,Current
4,10400.0,60 months,22.45,F,major_purchase,104433.0,25.37,1.0,3.0,MORTGAGE,...,Jun-1998,12.0,0.0,21929.0,64.5,35.0,Source Verified,Individual,PA,Fully Paid


## Target Variables

In [5]:
df['loan_status'].value_counts()

,count
loan_status,
Fully Paid,1076751
Current,878317
Charged Off,268559
Late (31-120 days),21467
In Grace Period,8436
Late (16-30 days),4349
Does not meet the credit policy. Status:Fully Paid,1988
Does not meet the credit policy. Status:Charged Off,761
Default,40


**Target feature change into Binary**

In [19]:
# Create a binary target variable: 1 for default, 0 for non-default
bad_statuses = ['Charged Off', 'Default']
df['is_default'] = df['loan_status'].apply(lambda x: 1 if x in bad_statuses else 0)

# Filter for only loans with a resolved outcome
resolved_statuses = ['Fully Paid'] + bad_statuses
df_cleaned = df[df['loan_status'].isin(resolved_statuses)].copy()
df_cleaned = df_cleaned.drop('loan_status', axis=1)

print(f"Original shape: {df.shape}")
print(f"Shape after filtering for resolved loans: {df_cleaned.shape}")
df['is_default'].value_counts()

Original shape: (1345383, 24)
Shape after filtering for resolved loans: (1345383, 23)


,count
is_default,
0,1076784
1,268599


## Missing Values

In [20]:
# Function to fill missing values
def fill_missing_values(df):
  """
    Fills missing values in a DataFrame based on column data type.

    Args:
      df: The input pandas DataFrame.

    Returns:
      The DataFrame with missing values filled.
  """
  for col in df.columns:
    if df[col].isnull().any():
      if df[col].dtype in ['int64', 'float64']:
        mean_val = df[col].mean()
        df[col].fillna(mean_val, inplace=True)
      else:
        mode_val = df[col].mode()[0]
        df[col].fillna(mode_val, inplace=True)
  return df

df = fill_missing_values(df_cleaned)

print("Missing values after filling:")
print(df.isna().sum())

Missing values after filling:
loan_amnt                0
term                     0
int_rate                 0
grade                    0
purpose                  0
annual_inc               0
dti                      0
delinq_2yrs              0
inq_last_6mths           0
home_ownership           0
emp_length               0
fico_range_low           0
fico_range_high          0
open_acc                 0
pub_rec                  0
revol_bal                0
revol_util               0
total_acc                0
verification_status      0
application_type         0
addr_state               0
is_default               0
credit_history_length    0
dtype: int64


## Date time/Category Values

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1345383 entries, 0 to 2260700
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   loan_amnt              1345383 non-null  float64
 1   term                   1345383 non-null  int64  
 2   int_rate               1345383 non-null  float64
 3   grade                  1345383 non-null  object 
 4   purpose                1345383 non-null  object 
 5   annual_inc             1345383 non-null  float64
 6   dti                    1345383 non-null  float64
 7   delinq_2yrs            1345383 non-null  float64
 8   inq_last_6mths         1345383 non-null  float64
 9   home_ownership         1345383 non-null  object 
 10  emp_length             1345383 non-null  int64  
 11  fico_range_low         1345383 non-null  float64
 12  fico_range_high        1345383 non-null  float64
 13  open_acc               1345383 non-null  float64
 14  pub_rec                

In [22]:
df.select_dtypes(include=['object']).head()

,grade,purpose,home_ownership,verification_status,application_type,addr_state
0,C,debt_consolidation,MORTGAGE,Not Verified,Individual,PA
1,C,small_business,MORTGAGE,Not Verified,Individual,SD
2,B,home_improvement,MORTGAGE,Not Verified,Joint App,IL
4,F,major_purchase,MORTGAGE,Source Verified,Individual,PA
5,C,debt_consolidation,RENT,Source Verified,Individual,GA


**Convert dates to date-time type**

In [ ]:
# 4. Engineer credit history length
# Check if the original date columns exist
if 'issue_d' in df.columns and 'earliest_cr_line' in df.columns:
    print("Engineering 'credit_history_length'...")
    # Ensure columns are datetime before performing operations
    df['issue_d'] = pd.to_datetime(df['issue_d'])
    df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
    df['credit_history_length'] = ((df['issue_d'] - df['earliest_cr_line']).dt.days / 30).round(0)
    # Drop original date columns now that we're done with them
    df = df.drop(columns=['issue_d', 'earliest_cr_line'], errors='ignore')

df['credit_history_length'].head()


Engineering 'credit_history_length'...


/tmp/ipython-input-10-1201896834.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['issue_d'] = pd.to_datetime(df['issue_d'])
/tmp/ipython-input-10-1201896834.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])


,credit_history_length
0,150.0
1,195.0
2,187.0
3,88.0
4,213.0


**Strip leading/tailing white spaces**

In [11]:
df = df.apply(lambda col: col.str.strip() if col.dtypes == 'object' else col)

In [12]:
df['emp_length'].value_counts()

,count
emp_length,
10+ years,894945
2 years,203677
< 1 year,189988
3 years,180753
1 year,148403
5 years,139698
4 years,136605
6 years,102628
7 years,92695


**Stripping suffix and signs in 'term' and 'emp_length' column**

In [13]:
df['term'] = pd.to_numeric(df['term'].astype(str).str[:2].str.strip(), errors='coerce')
df['emp_length'] = pd.to_numeric(df['emp_length'].str.replace('<', '', regex=False).str[:2].str.strip(), errors='coerce')

In [14]:
df['addr_state'].value_counts()

,count
addr_state,
CA,314566
NY,186389
TX,186335
FL,161991
IL,91173
NJ,83132
PA,76939
OH,75132
GA,74196


In [23]:
def value_counter(df):
    value_series = pd.DataFrame()
    for col in df.select_dtypes(["object"]).columns:
        print(df[col].value_counts(dropna = False))

value_counter(df)

grade
B    392781
C    381694
A    235095
D    200966
E     93656
F     32059
G      9132
Name: count, dtype: int64
purpose
debt_consolidation    780375
credit_card           295285
home_improvement       87507
other                  77877
major_purchase         29427
medical                15556
small_business         15416
car                    14588
moving                  9480
vacation                9065
house                   7254
wedding                 2294
renewable_energy         933
educational              326
Name: count, dtype: int64
home_ownership
MORTGAGE    665629
RENT        534436
OWN         144840
ANY            286
OTHER          144
NONE            48
Name: count, dtype: int64
verification_status
Source Verified    521322
Verified           418352
Not Verified       405709
Name: count, dtype: int64
application_type
Individual    1319577
Joint App       25806
Name: count, dtype: int64
addr_state
CA    196562
TX    110173
NY    109849
FL     95611
IL     51723
NJ

In [24]:
# In home_ownership column, 'ANY,OTHER and NONE' do not make sense to be alone
# themselves so that will group into 'OTHER'
home_ownership_mapping = {
    'ANY' : 'other',
    'MORTGAGE' : 'mortgage',
    'RENT' : 'rent',
    'OWN' : 'own',
    'NONE' : 'other'
}
df.home_ownership = df.home_ownership.map(home_ownership_mapping)

In [25]:
# 3. Engineer a single FICO score
# Check if the original FICO columns still exist before trying to create the new one
if 'fico_range_low' in df.columns and 'fico_range_high' in df.columns:
    print("Engineering 'fico_score'...")
    df['fico_score'] = (df['fico_range_low'] + df['fico_range_high']) / 2
    df = df.drop(columns=['fico_range_low', 'fico_range_high'])

df['fico_score'].head()
print(df['fico_score'].dtypes)

Engineering 'fico_score'...
float64


## All to Numeric Features

In [26]:
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(exclude=['object']).drop('is_default',axis=1).columns

# Concatenate numeric columns explicitly
df_final = pd.concat([df[num_cols], pd.get_dummies(df[cat_cols], drop_first=True), df['is_default']], axis=1)
df_final.head()

,loan_amnt,term,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,emp_length,open_acc,pub_rec,...,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,is_default
0,3600.0,36,13.99,55000.0,5.91,0.0,1.0,10,7.0,0.0,...,False,False,False,False,False,False,False,False,False,0
1,24700.0,36,11.99,65000.0,16.06,1.0,4.0,10,22.0,0.0,...,False,False,False,False,False,False,False,False,False,0
2,20000.0,60,10.78,63000.0,10.78,0.0,0.0,10,6.0,0.0,...,False,False,False,False,False,False,False,False,False,0
4,10400.0,60,22.45,104433.0,25.37,1.0,3.0,3,12.0,0.0,...,False,False,False,False,False,False,False,False,False,0
5,11950.0,36,13.44,34000.0,10.20,0.0,0.0,4,5.0,0.0,...,False,False,False,False,False,False,False,False,False,0
